In [1]:
import mackinac as mk
import pandas as pd
import joblib
from joblib import Parallel, delayed
from request_functions import *

#Set current ModelSEED client information
mk.modelseed.ms_client.url = 'https://p3.theseed.org/services/ProbModelSEED/'
mk.workspace.ws_client.url = 'https://p3.theseed.org/services/Workspace'
mk.genome.patric_url = 'https://user.patricbrc.org/authenticate'

#Obtain token; get_token will prompt password input
username = input("Input ModelSEED username: ")
mk.get_token(username)

#Load metadata from csv file
metadata = pd.read_csv("Ecoli_metadata.csv", delimiter=',')
metadata = metadata.drop([0])

#Note models which already exist in ModelSEED workspace
models = mk.list_modelseed_models()
model_ref = list((model['ref']) for model in models)
constructed_model_ids = []
for model in model_ref:
    model_id = model.split("modelseed/")[1]
    constructed_model_ids.append(model_id)

#Construct new models
model_indices = metadata.index.values.tolist()
count= 0
for i in model_indices:
    if metadata.at[i,'Genome ID'] not in constructed_model_ids:
        print("Constructing " + str(metadata.at[i,'Genome ID']))
        run_with_limited_time(construct_models, (metadata, i), {}, 3)
    else:
        #print("Genome " + str(metadata.at[i,'Genome ID']) + " already constructed")
        count = count+1
        if count == len(model_indices):
            print("All models in metadata were previously constructed.")
        pass
    
#Get model information - stoichiometric matrices and metabolite info - from construccted models
#Parallelize according to number of available cores - INPUT MUST BE INTEGER
cores = int(input("Cores available: "))
parent_directory = os.getcwd()
with joblib.parallel_backend(backend="multiprocessing"):
    results = Parallel(n_jobs=cores, verbose = 30, timeout = 30000)(delayed(get_model_info)(constructed_model_ids,i, parent_directory) for i in range(0,len(constructed_model_ids)))

Input ModelSEED username:  victoria.oceanna
patric password:  ··········


All models in metadata were previously constructed.


Cores available:  3


[Parallel(n_jobs=3)]: Using backend MultiprocessingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed:   10.2s
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   10.4s
[Parallel(n_jobs=3)]: Done   3 tasks      | elapsed:   10.8s
[Parallel(n_jobs=3)]: Done   4 tasks      | elapsed:   20.0s
[Parallel(n_jobs=3)]: Done   5 tasks      | elapsed:   20.2s
[Parallel(n_jobs=3)]: Done   6 tasks      | elapsed:   20.4s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   29.3s
[Parallel(n_jobs=3)]: Done   8 tasks      | elapsed:   29.7s
[Parallel(n_jobs=3)]: Done   9 tasks      | elapsed:   29.8s
[Parallel(n_jobs=3)]: Done  10 tasks      | elapsed:   38.3s
[Parallel(n_jobs=3)]: Done  11 tasks      | elapsed:   38.9s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   38.9s
[Parallel(n_jobs=3)]: Done  13 tasks      | elapsed:   48.0s
[Parallel(n_jobs=3)]: Done  14 tasks      | elapsed:   48.4s
[Parallel(n_jobs=3)]: Done  15 tasks      | elapsed:   48.5s

KeyboardInterrupt: 